In [4]:
!wget https://oldwebsite.techsimplus.com/media/Mask_noMask.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!unzip Mask_noMask.zip

Archive:  Mask_noMask.zip
replace dataset/with_mask/0-with-mask.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model


In [ ]:
with_mask_files = os.listdir('/content/dataset/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

['117-with-mask.jpg', '72-with-mask.jpg', 'augmented_image_41.jpg', '376-with-mask.jpg', 'augmented_image_34.jpg']
['239-with-mask.jpg', '200-with-mask.jpg', 'augmented_image_65.jpg', 'augmented_image_72.jpg', '418-with-mask.jpg']


In [ ]:
without_mask_files = os.listdir('/content/dataset/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

['augmented_image_41.jpg', '18.jpg', 'augmented_image_2.jpg', '456.jpg', '24.jpg']
['111.jpg', '162.jpg', '421.jpg', 'augmented_image_65.jpg', 'augmented_image_72.jpg']


In [ ]:
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))

Number of with mask images: 690
Number of without mask images: 686


In [ ]:

with_mask_labels = [1]*690

without_mask_labels = [0]*686

In [ ]:
labels = with_mask_labels + without_mask_labels

print(len(labels))
print(labels[0:5])
print(labels[-5:])

1376
[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]


In [ ]:


with_mask_path = '/content/dataset/with_mask/'
without_mask_path = '/content/dataset/without_mask/'

with_mask_files = os.listdir(with_mask_path)
without_mask_files = os.listdir(without_mask_path)

data = []
labels = []

for img_file in with_mask_files:
    image = Image.open(with_mask_path + img_file)
    image = image.resize((224, 224))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
    labels.append(1)  # 1 for with_mask

for img_file in without_mask_files:
    image = Image.open(without_mask_path + img_file)
    image = image.resize((224, 224))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
    labels.append(0)  # 0 for without_mask

data = np.array(data)
labels = np.array(labels)



In [ ]:
labels = to_categorical(labels, num_classes=2)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_reshaped = X_train.reshape((X_train.shape[0], -1))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

X_train_scaled = X_train_scaled.reshape((X_train.shape[0], 224, 224, 3))
X_test_scaled = X_test_scaled.reshape((X_test.shape[0], 224, 224, 3))

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation (if not done)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

datagen.fit(X_train_scaled)  # Apply to training data only

# Model Configuration
num_of_classes = 2  # For binary classification

model = keras.Sequential()

# Layer 1
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))  # Regularize with dropout

# Layer 2
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))

# Layer 3
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))

# Layer 4
model.add(keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

# Flatten and Fully Connected Layers
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))

# Output Layer
model.add(keras.layers.Dense(num_of_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.00005)  # Reduced learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 222, 222, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 109, 109, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 52, 52, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 24, 24, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 36864)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │      18,874,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │              

 Total params: 19,333,698 (73.75 MB)

 Trainable params: 19,331,458 (73.74 MB)

 Non-trainable params: 2,240 (8.75 KB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',  # Stop training when validation loss has stopped improving
    patience=5,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restores the best model after stopping
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',   # Reduces learning rate when validation loss stops improving
    factor=0.5,           # Factor by which to reduce the learning rate
    patience=3,           # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6           # Lower bound for the learning rate
)

model_checkpoint = ModelCheckpoint(
    'best_model.h5',     # Save the best model during training
    monitor='val_accuracy',  # Monitors validation accuracy to save the model with the best accuracy
    mode='max',          # Looks for the maximum validation accuracy to determine the best model
    save_best_only=True  # Only saves models with improved validation accuracy
)

# Fit the model
history = model.fit(
    datagen.flow(X_train_scaled, y_train, batch_size=32),
    epochs=50,  # Use a higher epoch count with early stopping to allow more training if beneficial
    validation_data=(X_test_scaled, y_test),
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1  # Display progress per epoch for tracking
)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=best_model.h5

In [ ]:
loaded_model=load_model('maskdetection.h5')

In [ ]:
loaded_model.summary()

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print('Test Accuracy =', accuracy)


In [ ]:
len(data)

In [ ]:
data[1]

In [ ]:
type(data[0])

In [ ]:
# converting image list and label list to numpy arrays

X = np.array(data)
Y = np.array(labels)

In [ ]:
type(X)

In [ ]:
type(Y)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
print(Y)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
# scaling the data

X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [ ]:
X_train[0]

In [ ]:
X_train_scaled[0]

In [ ]:
!wget -O facewm3.jpg https://img.freepik.com/free-photo/smiling-looking-front-young-beautiful-girl-wearing-brown-turtleneck-sweater-isolated-purple-wall_141793-90387.jpg?uid=R130571650&semt=ais_hybrid

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Load the pre-trained mask detection model
model = load_model('maskdetection.h5')

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Access the camera feed
cap = cv2.VideoCapture(0)  # 0 is for default camera

while True:
    ret, frame = cap.read()  # Capture frame-by-frame
    if not ret:
        print("Failed to grab frame")
        break

    # Convert to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y+h, x:x+w]

        # Preprocess the face for mask detection model
        face_resized = cv2.resize(face, (224, 224))
        face_normalized = face_resized / 255.0
        face_reshaped = np.reshape(face_normalized, [1, 224, 224, 3])

        # Predict if the person is wearing a mask or not
        prediction = model.predict(face_reshaped)
        pred_label = np.argmax(prediction)

        # Draw a rectangle around the face and label it
        if pred_label == 1:  # If wearing a mask
            color = (0, 255, 0)  # Green rectangle
            label = "Mask"
        else:  # If not wearing a mask
            color = (0, 0, 255)  # Red rectangle
            label = "No Mask"

        # Draw the rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        # Display the label above the rectangle
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Show the frame using matplotlib
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title('Mask Detection')
    plt.axis('off')  # Hide axes
    plt.show()

    # Press 'q' to break the loop and exit (for headless environments, this will need a manual stop)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera
cap.release()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp maskdetection.h5 /content/drive/MyDrive/
